In [24]:
import folium
import pandas as pd

In [25]:
# define the world map
world_map = folium.Map()

# display world map
world_map

In [26]:
# Netherlands latitude and longitude values 52.1326° N, 5.2913° E
latitude = 52.1326
longitude = 5.2913

# Create map and display it
utr_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Display the map of San Francisco
utr_map

In [31]:
# Read Dataset 
import requests
import json
import pandas as pd
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query1 = """
[out:json];
area["ISO3166-1"="NL"][admin_level=2];
(node["leisure"="garden"](area);
 way["leisure"="garden"](area);
 rel["leisure"="garden"](area);
);
out center;
"""
response1 = requests.get(overpass_url, 
                        params={'data': overpass_query1})
data = response1.json()
import numpy as np
import matplotlib.pyplot as plt
# Collect coords into list
coords1 = []
for element in data['elements']:
  if element['type'] == 'node':
    lon = element['lon']
    lat = element['lat']
    coords1.append((lon, lat))
  elif 'center' in element:
    lon = element['center']['lon']
    lat = element['center']['lat']
    coords1.append((lon, lat))
    
df_garden = pd.DataFrame(coords1)


overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query2 = """
[out:json];
area["ISO3166-1"="NL"][admin_level=2];
(node["landuse"="forest"](area);
 way["landuse"="forest"](area);
 rel["landuse"="forest"](area);
);
out center;
"""
response2 = requests.get(overpass_url, 
                        params={'data': overpass_query2})
data2 = response2.json()

# Collect coords into list
coords2 = []
for element in data2['elements']:
  if element['type'] == 'node':
    lon = element['lon']
    lat = element['lat']
    coords2.append((lon, lat))
  elif 'center' in element:
    lon = element['center']['lon']
    lat = element['center']['lat']
    coords2.append((lon, lat))
    
df_forest = pd.DataFrame(coords2)

frames = [df_forest, df_garden]

df = pd.concat(frames)

df.to_csv("Map_data.csv",index=False)
df.head()

,0,1
0,5.486640,52.306800
1,4.697121,52.399321
2,4.685800,52.294053
3,5.471657,52.451181
4,6.642369,52.046881


In [32]:
# get the data in the cdata
data = df

# Instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# Loop data
for lat, lng, in zip(df[1], df[0]):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=8, # define how big you want the circle markers to be
            color='Blue',
            fill=True,
            fill_color='red',
            fill_opacity=0.4
        )
    )

# Add incidents to map
san_map = folium.Map(location=[latitude, longitude], zoom_start=12)
san_map.add_child(incidents)


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93 entries, 0 to 83
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       93 non-null     float64
 1   1       93 non-null     float64
dtypes: float64(2)
memory usage: 2.2 KB


In [36]:
# Let's feak the data about the number of people who are hiking.
people = [3,36,34]
people = np.repeat(people, 31)
df["num"]= people
df.head()

,0,1,num
0,5.486640,52.306800,3
1,4.697121,52.399321,3
2,4.685800,52.294053,3
3,5.471657,52.451181,3
4,6.642369,52.046881,3


In [39]:
from folium import plugins

# let's start again with a clean copy of the map of NL

san_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
people1 = plugins.MarkerCluster().add_to(san_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df[1], df[0], df["num"]):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(people1)

# add incidents to map
san_map.add_child(people1)